In [9]:
import os
import json
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, GridSearchCV
import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

path=!pwd
SCRIPT_DIR = os.path.realpath(path[0])
print(SCRIPT_DIR)

/home/etsu/projects/Data_Mining_in_Action_2018_Spring/industry/hw02


In [10]:
%matplotlib notebook

In [11]:
data_path=SCRIPT_DIR + '/../seminar02/HR.csv'
df = pd.read_csv(data_path)
target_ = 'left'
features_ = [c for c in df if c != target_]
target = np.array(df[target_])
data = np.array(df[features_])

In [12]:
unique, counts = np.unique(target, return_counts=True)
unique, counts

(array([0, 1]), array([9821, 5178]))

In [13]:
df.head()

,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
0,0.53,2,157,3,0,1,0
1,0.86,5,262,6,0,0,0
2,0.88,7,272,4,0,1,0
3,0.87,5,223,5,0,1,0
4,0.52,2,159,3,0,1,0


In [14]:
params = {
    "learning_rate": 0.1, 
    "max_depth": 5, 
    "n_estimators": 1000, 
    "min_child_weight": 3,
    "seed": 42
}

In [15]:
%%time
estimator = xgb.XGBClassifier()
baseline_score = np.mean(cross_val_score(
                    estimator, data, target,
                    scoring='accuracy', 
                    cv=3
                ))
print('Baseline score {}'.format(baseline_score))

Baseline score 0.779121224244849
CPU times: user 505 ms, sys: 21 µs, total: 505 ms
Wall time: 503 ms


In [16]:
%%time
params = {
    "learning_rate": 0.1, 
    "max_depth": 5, 
    "n_estimators": 1000, 
    "min_child_weight": 3,
    "seed": 42
}
estimator = xgb.XGBClassifier(**params)
score = np.mean(cross_val_score(
                    estimator, data, target,
                    scoring='accuracy', 
                    cv=3
                ))
print('Default score {} ({})'.format(score, score-baseline_score))

Default score 0.7659867306794692 (-0.013134493565379768)
CPU times: user 7.36 s, sys: 11.9 ms, total: 7.38 s
Wall time: 7.38 s


In [17]:
%%time
param_grid1 = {
    "n_estimators": range(50,90,1)
}

xgb1 = xgb.XGBClassifier(
    learning_rate=0.1,
    seed=42
)

grid1 = GridSearchCV(xgb1, param_grid1, scoring='accuracy', n_jobs=4, cv=3)
grid1.fit(data, target)

print('=====\n'
      'Best params {}\n'
      'Score {} ({})\n'
      '====='.format(
          grid1.best_params_,
          grid1.best_score_, grid1.best_score_ - baseline_score
      ))

=====
Best params {'n_estimators': 82}
Score 0.7796519767984532 (0.0005307525536042412)
=====
CPU times: user 431 ms, sys: 84.1 ms, total: 515 ms
Wall time: 4.72 s


In [18]:
%%time
param_grid2 = {
    'max_depth':range(1,6),
    'min_child_weight':range(1,6)
}

xgb2 = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=82,
    seed=42
)

grid2 = GridSearchCV(xgb2, param_grid2, scoring='accuracy', n_jobs=3, cv=3)
grid2.fit(data, target)

print('=====\n'
      'Best params {}\n'
      'Score {} ({})\n'
      '====='.format(
          grid2.best_params_,
          grid2.best_score_, grid2.best_score_ - baseline_score
      ))

=====
Best params {'max_depth': 5, 'min_child_weight': 3}
Score 0.7802520168011201 (0.0011307925562711318)
=====
CPU times: user 379 ms, sys: 56.3 ms, total: 435 ms
Wall time: 4.75 s


In [19]:
%%time
param_grid3 = {
    'gamma':[i/100.0 for i in range(0,5)]
}

xgb3 = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=82,
    max_depth=5,
    min_child_weight=3,
    seed=42
)

grid3 = GridSearchCV(xgb3, param_grid3, scoring='accuracy', n_jobs=3, cv=3)
grid3.fit(data, target)

print('=====\n'
      'Best params {}\n'
      'Score {} ({})\n'
      '====='.format(
          grid3.best_params_,
          grid3.best_score_, grid3.best_score_ - baseline_score
      ))

=====
Best params {'gamma': 0.01}
Score 0.7803186879125275 (0.0011974636676784778)
=====
CPU times: user 429 ms, sys: 47.8 ms, total: 476 ms
Wall time: 1.84 s


In [20]:
%%time
param_grid4 = {
    'subsample':[i/100.0 for i in range(85,100)],
    'colsample_bytree':[i/100.0 for i in range(75,85)]
}

xgb4 = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=82,
    max_depth=5,
    min_child_weight=3,
    gamma=0.1,
    seed=42
)

grid4 = GridSearchCV(xgb4, param_grid4, scoring='accuracy', n_jobs=3, cv=3)
grid4.fit(data, target)

print('=====\n'
      'Best params {}\n'
      'Score {} ({})\n'
      '====='.format(
          grid4.best_params_,
          grid4.best_score_, grid4.best_score_ - baseline_score
      ))

=====
Best params {'colsample_bytree': 0.84, 'subsample': 0.89}
Score 0.781652110140676 (0.002530885895827062)
=====
CPU times: user 1.18 s, sys: 603 ms, total: 1.78 s
Wall time: 39.2 s


In [21]:
%%time
param_grid5 = {
    'reg_alpha':[0, 1e-10, 1e-5, 1e-2]
}

xgb5 = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=82,
    max_depth=5,
    min_child_weight=3,
    gamma=0.1,
    colsample_bytree=0.75,
    subsample=0.97,
    seed=42
)

grid5 = GridSearchCV(xgb5, param_grid5, scoring='accuracy', n_jobs=3, cv=3)
grid5.fit(data, target)

print('=====\n'
      'Best params {}\n'
      'Score {} ({})\n'
      '====='.format(
          grid5.best_params_,
          grid5.best_score_, grid5.best_score_ - baseline_score
      ))

=====
Best params {'reg_alpha': 0.01}
Score 0.7809187279151943 (0.0017975036703453684)
=====
CPU times: user 419 ms, sys: 27.9 ms, total: 446 ms
Wall time: 1.61 s


In [22]:
%%time
param_grid6 = {
    'n_estimators': range(62,65),
    'max_depth': [6,7,8,9,10],
    'min_child_weight': [2,3,4],
}

xgb6 = xgb.XGBClassifier(
    learning_rate=0.1,
    gamma=0.1,
    colsample_bytree=0.75,
    subsample=0.97,
    seed=42
)

grid6 = GridSearchCV(xgb6, param_grid6, scoring='accuracy', n_jobs=3, cv=3)
grid6.fit(data, target)

print('=====\n'
      'Best params {}\n'
      'Score {} ({})\n'
      '====='.format(
          grid6.best_params_,
          grid6.best_score_, grid6.best_score_ - baseline_score
      ))

=====
Best params {'max_depth': 7, 'min_child_weight': 3, 'n_estimators': 64}
Score 0.7833188879258617 (0.004197663681012709)
=====
CPU times: user 658 ms, sys: 193 ms, total: 851 ms
Wall time: 15.1 s


In [68]:
%%time
params = {
    "learning_rate": 0.001,
    "n_estimators": 500,
    "max_depth": 8, 
    "min_child_weight": 0,
    "gamma": 0.1,
    "colsample_bytree": 0.75,
    "subsample": 0.97,
    "seed": 42
}
estimator = xgb.XGBClassifier(**params)
score = np.mean(cross_val_score(
                    estimator, data, target,
                    scoring='accuracy', 
                    cv=3
                ))
print('Score {} ({})'.format(score, score-baseline_score))

Score 0.7845885177035408 (0.005467293458691791)
CPU times: user 6.3 s, sys: 4.89 ms, total: 6.31 s
Wall time: 6.31 s


In [67]:
%%time
param_grid7 = {
    "learning_rate": [0.001, 0.1],
    "n_estimators": range(480,520,1),
}
params = {
    "max_depth": 8, 
    "min_child_weight": 0,
    "gamma": 0.1,
    "colsample_bytree": 0.75,
    "subsample": 0.97,
    "seed": 42
}
xgb7 = xgb.XGBClassifier(**params)

grid7 = GridSearchCV(xgb7, param_grid7, scoring='accuracy', n_jobs=7, cv=3)
grid7.fit(data, target)

print('=====\n'
      'Best params {}\n'
      'Score {} ({})\n'
      '====='.format(
          grid7.best_params_,
          grid7.best_score_, grid7.best_score_ - baseline_score
      ))

=====
Best params {'learning_rate': 0.001, 'n_estimators': 500}
Score 0.7845856390426028 (0.005464414797753836)
=====
CPU times: user 3.42 s, sys: 436 ms, total: 3.85 s
Wall time: 2min 21s
